In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./archive (1)/annotations\Cars0.xml,226,419,125,173
1,./archive (1)/annotations\Cars1.xml,134,262,128,160
2,./archive (1)/annotations\Cars10.xml,140,303,5,148
3,./archive (1)/annotations\Cars100.xml,175,214,114,131
4,./archive (1)/annotations\Cars101.xml,167,240,202,220


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][0]
filename

'./archive (1)/annotations\\Cars0.xml'

In [5]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./archive (1)/images',filename_image)
    return filepath_image

In [6]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['./archive (1)/images\\Cars0.png',
 './archive (1)/images\\Cars1.png',
 './archive (1)/images\\Cars10.png',
 './archive (1)/images\\Cars100.png',
 './archive (1)/images\\Cars101.png',
 './archive (1)/images\\Cars102.png',
 './archive (1)/images\\Cars103.png',
 './archive (1)/images\\Cars104.png',
 './archive (1)/images\\Cars105.png',
 './archive (1)/images\\Cars106.png',
 './archive (1)/images\\Cars107.png',
 './archive (1)/images\\Cars108.png',
 './archive (1)/images\\Cars109.png',
 './archive (1)/images\\Cars11.png',
 './archive (1)/images\\Cars110.png',
 './archive (1)/images\\Cars111.png',
 './archive (1)/images\\Cars112.png',
 './archive (1)/images\\Cars113.png',
 './archive (1)/images\\Cars114.png',
 './archive (1)/images\\Cars115.png',
 './archive (1)/images\\Cars116.png',
 './archive (1)/images\\Cars117.png',
 './archive (1)/images\\Cars118.png',
 './archive (1)/images\\Cars119.png',
 './archive (1)/images\\Cars12.png',
 './archive (1)/images\\Cars120.png',
 './archive (1)/ima

In [7]:
# verify image and op 

In [8]:
file_path = image_path[1]
file_path 

'./archive (1)/images\\Cars1.png'

In [9]:
img = cv2.imread(file_path)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Data Preprocessing

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [11]:
labels = df.iloc[:,1:].values

In [12]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [13]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [14]:
X.shape,y.shape

((433, 224, 224, 3), (433, 4))

In [15]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.9,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((389, 224, 224, 3), (44, 224, 224, 3), (389, 4), (44, 4))

In [16]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [17]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [18]:
# complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

activation_125 (Activation)     (None, 12, 12, 128)  0           batch_normalization_125[0][0]    
__________________________________________________________________________________________________
conv2d_126 (Conv2D)             (None, 12, 12, 160)  143360      activation_125[0][0]             
__________________________________________________________________________________________________
batch_normalization_126 (BatchN (None, 12, 12, 160)  480         conv2d_126[0][0]                 
__________________________________________________________________________________________________
activation_126 (Activation)     (None, 12, 12, 160)  0           batch_normalization_126[0][0]    
__________________________________________________________________________________________________
conv2d_124 (Conv2D)             (None, 12, 12, 192)  208896      block17_12_ac[0][0]              
__________________________________________________________________________________________________
conv2d_127

conv2d_141 (Conv2D)             (None, 12, 12, 128)  139264      block17_16_ac[0][0]              
__________________________________________________________________________________________________
batch_normalization_141 (BatchN (None, 12, 12, 128)  384         conv2d_141[0][0]                 
__________________________________________________________________________________________________
activation_141 (Activation)     (None, 12, 12, 128)  0           batch_normalization_141[0][0]    
__________________________________________________________________________________________________
conv2d_142 (Conv2D)             (None, 12, 12, 160)  143360      activation_141[0][0]             
__________________________________________________________________________________________________
batch_normalization_142 (BatchN (None, 12, 12, 160)  480         conv2d_142[0][0]                 
__________________________________________________________________________________________________
activation

## Model Training

In [19]:
from tensorflow.keras.callbacks import TensorBoard

In [20]:
tfb = TensorBoard('object_detection')

In [21]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
39/39 [==============================] - 126s 3s/step - loss: 0.1437 - val_loss: 0.1352
Epoch 2/100
39/39 [==============================] - 88s 2s/step - loss: 0.1358 - val_loss: 0.1220
Epoch 3/100
39/39 [==============================] - 77s 2s/step - loss: 0.1298 - val_loss: 0.1254
Epoch 4/100
39/39 [==============================] - 73s 2s/step - loss: 0.1287 - val_loss: 0.1171
Epoch 5/100
39/39 [==============================] - 73s 2s/step - loss: 0.1276 - val_loss: 0.1181
Epoch 6/100
39/39 [==============================] - 74s 2s/step - loss: 0.1247 - val_loss: 0.1154
Epoch 7/100
39/39 [==============================] - 73s 2s/step - loss: 0.1259 - val_loss: 0.1165
Epoch 8/100
39/39 [==============================] - 72s 2s/step - loss: 0.1233 - val_loss: 0.1137
Epoch 9/100
39/39 [==============================] - 71s 2s/step - loss: 0.1197 - val_loss: 0.1168
Epoch 10/100
39/39 [==============================] - 71s 2s/step - loss: 0.1178 - val_loss: 0.1265
Epoch 11

Epoch 83/100
39/39 [==============================] - 87s 2s/step - loss: 0.0015 - val_loss: 0.0089
Epoch 84/100
39/39 [==============================] - 87s 2s/step - loss: 0.0014 - val_loss: 0.0085
Epoch 85/100
39/39 [==============================] - 89s 2s/step - loss: 0.0013 - val_loss: 0.0096
Epoch 86/100
39/39 [==============================] - 86s 2s/step - loss: 0.0012 - val_loss: 0.0090
Epoch 87/100
39/39 [==============================] - 87s 2s/step - loss: 0.0011 - val_loss: 0.0090
Epoch 88/100
39/39 [==============================] - 86s 2s/step - loss: 0.0012 - val_loss: 0.0086
Epoch 89/100
39/39 [==============================] - 85s 2s/step - loss: 0.0012 - val_loss: 0.0095
Epoch 90/100
39/39 [==============================] - 86s 2s/step - loss: 0.0015 - val_loss: 0.0090
Epoch 91/100
39/39 [==============================] - 86s 2s/step - loss: 0.0013 - val_loss: 0.0094
Epoch 92/100
39/39 [==============================] - 87s 2s/step - loss: 0.0013 - val_loss: 0.0092


In [ ]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
                 validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

In [23]:
model.save('./models/object_detection_final.h5')